In [15]:
# Snippet 1
import requests
import json
import folium
from functools import lru_cache
from shapely.geometry import LineString

# Please add your API key for Google Maps Directions API
API_KEY = ''

# Create a cache for route calculations
@lru_cache(maxsize=None)
def calculate_route(origin, destination):
    url = f'https://maps.googleapis.com/maps/api/directions/json?origin={origin[0]},{origin[1]}&destination={destination[0]},{destination[1]}&mode=walking&key={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data


# Read the data from the CSV file and extract the conflict zone and camp locations
conflict_zone_locations = []
camp_locations = []

with open('locations.csv', 'r') as file:
    lines = file.readlines()[1:]  # Skip the header line
    for line in lines:
        fields = line.strip().split(',')
        location_name = fields[0]
        location_coordinates = (float(fields[3]), float(fields[4]))
        location_type = fields[5]
        if location_type == 'conflict_zone':
            conflict_zone_locations.append((location_name, location_coordinates))
        elif location_type == 'camp':
            camp_locations.append((location_name, location_coordinates))

# Process each conflict zone
features = []
for zone1 in conflict_zone_locations + camp_locations:
    zone1_name, zone1_coordinates = zone1

    # Calculate routes from the current location to all other conflict zones and camps
    routes = []
    for zone2 in conflict_zone_locations + camp_locations:
        zone2_name, zone2_coordinates = zone2

        # Skip if the locations are the same
        if zone1_name == zone2_name:
            continue

        try:
            # Calculate the route or retrieve from cache
            data = calculate_route(zone1_coordinates, zone2_coordinates)

            # Check if the route is found
            if data['status'] == 'OK':
                # Extract the route coordinates
                coordinates = []
                for step in data['routes'][0]['legs'][0]['steps']:
                    start_location = step['start_location']
                    end_location = step['end_location']
                    coordinates.append((start_location['lat'], start_location['lng']))
                    coordinates.append((end_location['lat'], end_location['lng']))

                # Create a LineString object from the route coordinates
                route_line = LineString(coordinates)

                # Simplify the LineString by reducing the number of points
                tolerance = 0.001  # Adjust the tolorance "up" for more and "down" for fewrer coordinates 
                simplified_route = route_line.simplify(tolerance)

                # Extract the simplified coordinates from the LineString object
                simplified_coordinates = list(simplified_route.coords)

                routes.append({
                    'name': zone2_name,
                    'coordinates': simplified_coordinates
                })

            else:
                print(f"No route found between {zone1_name} and {zone2_name}")

        except Exception as e:
            print(f"Error occurred: {e}")

    # Add the routes as features to the GeoJSON
    if routes:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "MultiLineString",
                "coordinates": [route['coordinates'] for route in routes]
            },
            "properties": {
                "start": {
                    "name": zone1_name,
                    "coordinates": simplified_coordinates
                },
                "routes": routes
            }
        }
        features.append(feature)

# Create the GeoJSON object
geojson = {
    "type": "FeatureCollection",
    "features": features
}


# Printouts for total number of coordinates in GeoJson file and between routes 
total_coordinates = 0

# Iterate over the features and calculate the total number of coordinates
for feature in geojson['features']:
    coordinates = feature['geometry']['coordinates'][0]
    total_coordinates += len(coordinates)

print("Total number of coordinates in the GeoJSON file (excluding start/end):", total_coordinates)

# Iterate over the features and count the number of coordinates per route
for feature in geojson['features']:
    coordinates = feature['geometry']['coordinates'][0]
    num_coordinates = len(coordinates)
    print(f"Route: {feature['properties']['start']['name']} to {feature['properties']['routes'][0]['name']} with {num_coordinates} coordinates.")


# Save the GeoJSON to a file
with open("routes.geojson", "w") as file:
    json.dump(geojson, file)

print("GeoJSON file saved successfully.")

# Read the GeoJSON file
with open('routes.geojson') as f:
    data = json.load(f)

# Calculate the center location based on all coordinates
all_coordinates = [coord for _, coord in conflict_zone_locations + camp_locations]
m = folium.Map(location=all_coordinates[0], zoom_start=6)

# Iterate over the features and add them as PolyLine to the map
for feature in data['features']:
    coordinates = feature['geometry']['coordinates'][0]
    folium.PolyLine(locations=coordinates, color='blue').add_to(m)

# Add conflict zones as green circles
for zone in conflict_zone_locations:
    zone_name, zone_coordinates = zone
    folium.CircleMarker(location=zone_coordinates, 
                        radius=6, color='red', 
                        fill=True, fill_color='red').add_to(m)
    folium.Marker(
        location=zone_coordinates, 
        popup=zone_name,
        icon=folium.DivIcon(icon_size=(1,1))
    ).add_to(m)

# Add camps as red circles
for camp in camp_locations:
    camp_name, camp_coordinates = camp
    folium.CircleMarker(location=camp_coordinates, 
                        radius=6, color='green', 
                        fill=True, fill_color='green').add_to(m)
    folium.Marker(
        location=camp_coordinates,
        tooltip=camp_name,
        icon=folium.DivIcon(icon_size=(1,1))
    ).add_to(m)

# Save the map as an HTML file
m.save('map.html')

# Display the map
m


No route found between Gao and Segou
No route found between Gao and Tombouctou
No route found between Gao and Douentza
No route found between Gao and Kayes
No route found between Gao and Sikasso
No route found between Gao and Koulikoro
No route found between Gao and Menaka
No route found between Gao and Kidal
No route found between Gao and Bamako
No route found between Gao and Fassala
No route found between Gao and Mentao
No route found between Gao and Dori
No route found between Gao and Damba
No route found between Gao and Djibo
No route found between Gao and Niamey
No route found between Gao and Mangaize
No route found between Gao and Ayorou
No route found between Gao and Tahoua
No route found between Gao and Abala
No route found between Segou and Gao
No route found between Segou and Tombouctou
No route found between Segou and Douentza
No route found between Segou and Kayes
No route found between Segou and Sikasso
No route found between Segou and Koulikoro
No route found between Sego

No route found between Fassala and Kidal
No route found between Fassala and Bamako
No route found between Fassala and Mentao
No route found between Fassala and Dori
No route found between Fassala and Damba
No route found between Fassala and Djibo
No route found between Fassala and Niamey
No route found between Fassala and Mangaize
No route found between Fassala and Ayorou
No route found between Fassala and Tahoua
No route found between Fassala and Abala
No route found between Mentao and Gao
No route found between Mentao and Segou
No route found between Mentao and Tombouctou
No route found between Mentao and Douentza
No route found between Mentao and Kayes
No route found between Mentao and Sikasso
No route found between Mentao and Koulikoro
No route found between Mentao and Menaka
No route found between Mentao and Kidal
No route found between Mentao and Bamako
No route found between Mentao and Fassala
No route found between Mentao and Dori
No route found between Mentao and Damba
No rout

In [14]:
# Snippet 2
import requests
import json
import folium
from functools import lru_cache
from shapely.geometry import LineString
from multiprocessing import Pool, cpu_count

# API key for Google Maps Directions API
API_KEY = ''

# Create a cache for route calculations
@lru_cache(maxsize=None)
def calculate_route(origin, destination):
    url = f'https://maps.googleapis.com/maps/api/directions/json?origin={origin[0]},{origin[1]}&destination={destination[0]},{destination[1]}&mode=walking&key={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data

# Define a function to calculate the route for a given pair of locations
def calculate_route_for_pair(zone1, zone2):
    zone1_name, zone1_coordinates = zone1
    zone2_name, zone2_coordinates = zone2

    # Skip if the locations are the same
    if zone1_name == zone2_name:
        return None

    try:
        # Calculate the route or retrieve from cache
        data = calculate_route(zone1_coordinates, zone2_coordinates)

        # Check if the route is found
        if data['status'] == 'OK':
            # Extract the route coordinates
            coordinates = []
            for step in data['routes'][0]['legs'][0]['steps']:
                start_location = step['start_location']
                end_location = step['end_location']
                coordinates.append((start_location['lat'], start_location['lng']))
                coordinates.append((end_location['lat'], end_location['lng']))

            # Create a LineString object from the route coordinates
            route_line = LineString(coordinates)

            # Simplify the LineString by reducing the number of points
            tolerance = 0.0001  # Adjust the tolerance "up" for more and "down" for fewer coordinates
            simplified_route = route_line.simplify(tolerance)

            # Extract the simplified coordinates from the LineString object
            simplified_coordinates = list(simplified_route.coords)

            return {
                'name': zone2_name,
                'coordinates': simplified_coordinates
            }

        else:
            print(f"No route found between {zone1_name} and {zone2_name}")

    except Exception as e:
        print(f"Error occurred: {e}")

    return None

# Read the data from the CSV file and extract the conflict zone and camp locations
conflict_zone_locations = []
camp_locations = []

with open('locations.csv', 'r') as file:
    lines = file.readlines()[1:]
    for line in lines:
        fields = line.strip().split(',')
        location_name = fields[0]
        location_coordinates = (float(fields[3]), float(fields[4]))
        location_type = fields[5]
        if location_type == 'conflict_zone':
            conflict_zone_locations.append((location_name, location_coordinates))
        elif location_type == 'camp':
            camp_locations.append((location_name, location_coordinates))

# Process each conflict zone
features = []
num_locations = len(conflict_zone_locations + camp_locations)
num_pairs = num_locations * (num_locations - 1)

# Create a list of location pairs for parallel processing
location_pairs = []
for zone1 in conflict_zone_locations + camp_locations:
    for zone2 in conflict_zone_locations + camp_locations:
        location_pairs.append((zone1, zone2))


# Define a function to calculate routes for a batch of location pairs
def calculate_routes_batch(location_pairs):
    routes = []
    for zone1, zone2 in location_pairs:
        route = calculate_route_for_pair(zone1, zone2)
        if route:
            routes.append(route)
    return routes


# Split the location pairs into chunks for parallel processing
num_processes = cpu_count()
chunk_size = num_pairs // num_processes
chunks = [location_pairs[i:i+chunk_size] for i in range(0, num_pairs, chunk_size)]

# Create a multiprocessing Pool and map the chunks to worker processes
with Pool(processes=num_processes) as pool:
    results = pool.map(calculate_routes_batch, chunks)

# Flatten the results from all processes
routes = [route for result in results for route in result if route]

# Add the routes as features to the GeoJSON
for zone1 in conflict_zone_locations + camp_locations:
    zone1_name, zone1_coordinates = zone1
    zone_routes = [route for route in routes if route['name'] == zone1_name]
    if zone_routes:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "MultiLineString",
                "coordinates": [route['coordinates'] for route in zone_routes]
            },
            "properties": {
                "start": {
                    "name": zone1_name,
                    "coordinates": zone1_coordinates
                },
                "routes": zone_routes
            }
        }
        features.append(feature)

# Create the GeoJSON object
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON to a file
with open("routes.geojson", "w") as file:
    json.dump(geojson, file)

print("GeoJSON file saved successfully.")

# Read the GeoJSON file
with open('routes.geojson') as f:
    data = json.load(f)

# Calculate the center location based on all coordinates
all_coordinates = [coord for _, coord in conflict_zone_locations + camp_locations]
m = folium.Map(location=all_coordinates[0], zoom_start=6)

# Iterate over the features and add them as PolyLine to the map
for feature in data['features']:
    coordinates = feature['geometry']['coordinates'][0]
    folium.PolyLine(locations=coordinates, color='blue').add_to(m)

# Add conflict zones as green circles
for zone in conflict_zone_locations:
    zone_name, zone_coordinates = zone
    folium.CircleMarker(location=zone_coordinates, 
                        radius=6, color='red', 
                        fill=True, fill_color='red').add_to(m)
    folium.Marker(
        location=zone_coordinates, 
        popup=zone_name,
        icon=folium.DivIcon(icon_size=(1,1))
    ).add_to(m)

# Add camps as red circles
for camp in camp_locations:
    camp_name, camp_coordinates = camp
    folium.CircleMarker(location=camp_coordinates, 
                        radius=6, color='green', 
                        fill=True, fill_color='green').add_to(m)
    folium.Marker(
        location=camp_coordinates,
        tooltip=camp_name,
        icon=folium.DivIcon(icon_size=(1,1))
    ).add_to(m)

# Save the map as an HTML file
m.save('map.html')

# Display the map
m

GeoJSON file saved successfully.
